# Electronic Structure - Basic Structure/Procedure

#### Born-Oppenheimer approximation

In molecules, nuclei are much heavier than electrons, thus they don't move on same time scale; therefore the behaviour of nuclei and electrons can be decoupled.

Therefore, one can first tackle the electronic problem with nuclear coordinates entering only as parameters. The energy levels of the electrons in the molecule can be found by solving the non-relativistic time-independent Schroedinger equation,

$
\begin{align}
\mathcal{H}_{el} |\Psi_n \rangle = E_n |\Psi_n \rangle
\end{align}
$

The ground state energy is given by:

$
\begin{align}
E_0 = \frac{ \langle\Psi_0|H_{el}|\Psi_0\rangle } { \langle\Psi_0|\Psi_0\rangle }
\end{align}
$

We would like to prepare the ground state, $\Psi_0$ on a quantum computer and measure the Hamiltonian expectation value, $E_0$ directly.

### The Hartree-Fock initial state

Good starting point to this problem is the Hartree-Fock method. This method approximates a N-body problem into N one-body problems where each electron evolves in the mean-field of other.

The Hamiltonian can then be expressed in the basis of the solutions of the HF method, also called Molecular Orbitals (MOs).

MOs ($\phi_\mu$) can be occupied or virtual (unoccupied). One MO can contain 2 electrons. For now we work with Spin Orbitals which are associated with a spin up ($\alpha$) or spin down ($\beta$) electron. Thus spin orbitals can contain one electron or be un-occupied.

There are different codes able to find HF solutions: **Gaussian, Psi4, PyQuante, PySCF**

Below we set up a PySCF driver for H2 molecule at equilibrium bond length 0.735 Angstrom, in the singlet state and with no charge.

In [1]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)

molecule = Molecule(
    geometry = [
        ["H", [0.0, 0.0, 0.0]],
        ["H", [0.0, 0.0, 0.735]]],
    charge = 0,
    multiplicity = 1
)

driver = ElectronicStructureMoleculeDriver(
    molecule,
    basis = "sto3g",
    driver_type = ElectronicStructureDriverType.PYSCF
)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Molecular Hamiltonian is expressed in terms of fermionic operators and these operators mmust be mapped to spin operators to be Qubit Hamiltonian.

Different mapping types: Jordan-Wigner mapping, Parity mapping, Bravyi-Kitaev mapping.

JW mapping usu maps each Spin Orbital to a qubit.

Below, we set up Electronic Structure Problem to generate the Second quantized operator and a qubit converter that will map it to a qubit operator.

In [2]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

In [3]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

Fermionic Operator
register length=4, number terms=14
  (0.18093119978423106+0j) * ( +_0 -_1 +_2 -_3 )
+ (-0.18093119978423128+0j) * ( +_0 -_1 -_2 +_3 )
+ (-0.18093119978423128+0j) * ( -_0 +_1 +_2 -_3 )
+ (0.18093119978423144+0j) * ( -_0 +_1 -_2 +_3  ...


Transform this Hamiltonian for given driver defined above we get our qubit operator:

In [4]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

0.04523279994605781 * YYYY
+ 0.04523279994605781 * XXYY
+ 0.04523279994605781 * YYXX
+ 0.04523279994605781 * XXXX
- 0.8105479805373281 * IIII
- 0.22575349222402358 * ZIII
+ 0.17218393261915543 * IZII
+ 0.12091263261776633 * ZZII
- 0.22575349222402358 * IIZI
+ 0.17464343068300436 * ZIZI
+ 0.16614543256382414 * IZZI
+ 0.17218393261915543 * IIIZ
+ 0.16614543256382414 * ZIIZ
+ 0.16892753870087904 * IZIZ
+ 0.12091263261776633 * IIZZ


In the minimal (STO-3G) basis set 4 qubits are required. We can reduce number of qubits by using Parity mapping, which allows for removal of 2 qubits by exploiting known symmetries arising from the mapping.

In [5]:
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
print(qubit_op)

0.18093119978423114 * XX
- 1.0523732457728605 * II
- 0.39793742484317884 * ZI
+ 0.39793742484317884 * IZ
- 0.011280104256235171 * ZZ


This time only two qubits are required.

The Hamiltonian is ready and can be used in a quantum algorithm to find into about electronic structure of corresponding molecule.